In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

In [2]:
import time
import pandas as pd
from io import StringIO

def scrape_url(year, week):
    driver.get(url=f"https://www.fantasypros.com/nfl/rankings/k.php?year={year}&week={week}")
    time.sleep(5)  # give time for page to load

    data = pd.read_html(StringIO(driver.page_source))[0]
    data = data[['RK', 'Player Name']]  # opponent and matchup data is innacurate

    # get rid of team from player name
    data['Player Name'] = data['Player Name'].str.split(' \(').str[0]

    data.insert(0, "Year", year)
    data.insert(1, "Week", week)

    return data

# scrape_url(2023, 1)

,Year,Week,RK,Player Name
0,2023,1,1,Justin Tucker
1,2023,1,2,Harrison Butker
2,2023,1,3,Jason Myers
3,2023,1,4,Younghoe Koo
4,2023,1,5,Tyler Bass
5,2023,1,6,Evan McPherson
6,2023,1,7,Brandon McManus
7,2023,1,8,Daniel Carlson
8,2023,1,9,Jason Sanders
9,2023,1,10,Greg Joseph


In [3]:
from itertools import product
from tqdm.autonotebook import tqdm

d = []
for year, week in tqdm(list(product([year for year in range(2018, 2024)], [week for week in range(1, 18)]))):
    if (week != 17) or (year >= 2021):
        d.append(scrape_url(year, week))

C:\Users\mhugh\AppData\Local\Temp\ipykernel_14732\1166229690.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/102 [00:00<?, ?it/s]

In [4]:
df = pd.concat(d, ignore_index=True)

df.insert(df.columns.get_loc('Player Name') + 1, 'POS', 'K')

df.rename(columns={'RK': 'POS RK'}, inplace=True)

df

,Year,Week,RK,Player Name,POS
0,2018,1,1,Justin Tucker,K
1,2018,1,2,Stephen Gostkowski,K
2,2018,1,3,Greg Zuerlein,K
3,2018,1,4,Wil Lutz,K
4,2018,1,5,Matt Prater,K
...,...,...,...,...,...
3104,2023,17,29,Eddy Pineiro,K
3105,2023,17,30,Greg Zuerlein,K
3106,2023,17,31,Joey Slye,K
3107,2023,17,32,Chad Ryland,K


In [8]:
df.groupby(['Year', 'Week'])['Player Name'].count().min()

np.int64(26)

In [10]:
df.to_parquet(r'..\..\data\fantasypros_in_season_rankings\kicker_rankings.parquet')

'Done'

'Done'